In [ ]:
# ===== CONSTANTS ===== 
# Legt den Dateinamen für die Zwischenspeicherung (Cache) fest
cache_file = "cache.csv"

# ===== CORE FUNCTIONS =====

"""
Liеst eine CSV-Datei ein und trennt sie in Header (Spaltennamen) und Zeilen auf.

Parameter:
  file (str): Pfad zur CSV-Datei, die eingelesen werden soll.

Rückgabe:
  header (list): Liste der Spaltennamen aus der ersten Zeile.
  lines (list): Liste aller Zeilen der Datei als rohe Textzeilen.

Funktionsablauf:
  1. Öffnet die Datei im Lesemodus.
  2. Liest alle Zeilen in eine Liste ein.
  3. Trennt die erste Zeile vom Rest und zerlegt sie in Spaltennamen.
"""
def get_content(file):
    # Öffnet die Datei und liest alle Zeilen
    with open(file, "r") as data:
        lines = data.readlines()  # Jede Zeile als String inklusive Zeilenumbruch
        # Entfernt Leerzeichen und Zeilenumbruch, dann split an Komma
        header = lines[0].strip().split(",")
    # Gibt Header und alle gelesenen Zeilen zurück
    return header, lines

"""
Erzeugt oder überschreibt die Cache-Datei mit gegebenem Header und Zeilen.

Parameter:
  header (list): Liste von Spaltennamen, wird als erste Zeile gespeichert.
  lines (list of lists): Jede Zeile bereits als Liste von Werten.
  file (str): Dateiname für die Ausgabedatei (Standard: 'cache.csv').

Funktionsablauf:
  1. Öffnet (oder erstellt) die Datei im Schreibmodus.
  2. Schreibt den Header als kommagetrennte Zeile.
  3. Iteriert durch die Zeilen und schreibt jede als kommagetrennte Werte.
"""
def generate_cache_file(header, lines, file=cache_file):
    with open(file, "w") as new_file:
        # Header schreiben
        new_file.write(",".join(header) + "\n")
        # Jede Datenzeile schreiben
        for line in lines:
            new_file.write(",".join(line) + "\n")

"""
Gibt eine formatierte Tabelle auf der Konsole aus.

Parameter:
  file (str): Pfad zur CSV-Datei, die angezeigt werden soll.

Funktionsablauf:
  1. Liest Header und Zeilen mit get_content().
  2. Berechnet Anzahl der Datenzeilen und Gesamttabellenbreite.
  3. Zeichnet Trennlinien und druckt Header mit fester Spaltenbreite (20 Zeichen).
  4. Druckt jede Zeile formatiert.

Inspiration & Quelle:
  Flexible Tabellenformatierung in Python –
  https://stackoverflow.com/questions/70937491/python-flexible-way-to-format-string-output-into-a-table-without-using-a-non-st
"""
def print_formatted_table(file=cache_file):
    header, lines = get_content(file)
    # Überschrift anzeigen
    print("\nFormatted Table, from file:", file)
    # Anzahl der Datenzeilen (ohne Header)
    print(f"With a total of {len(lines)-1} entrys.")
    # Horizontale Trennlinie basierend auf Spaltenanzahl
    print("-" * round(23.2 * len(header)))
    # Headerzeile, jede Spalte 20 Zeichen breit, linksbündig
    print(("| {:<20} " * len(header) + "|").format(*header))
    print("-" * round(23.2 * len(header)))
    # Jede Datenzeile drucken
    for line in lines[1:]:
        values = line.strip().split(",")
        print(("| {:<20} " * len(header) + "|").format(*values))

"""
Filtert Zeilen nach einem bestimmten Spaltenwert und schreibt das Ergebnis in die Cache-Datei.

Parameter:
  column (str): Spaltenname, nach dem gefiltert werden soll.
  value (list of str): Liste der gewünschten Werte.
  file (str): Pfad zur Eingabedatei.
  return_table (bool): Wenn True, wird die gefilterte Tabelle direkt angezeigt.

Funktionsablauf:
 1. Lese Header und Zeilen.
 2. Prüfe, ob die Spalte existiert.
 3. Iteriere über alle Datenzeilen, teile sie in Werte auf.
 4. Falls der Wert in der gewünschten Liste, füge zur Ergebnisliste hinzu.
 5. Schreibe gefilterte Zeilen in Cache-Datei.
 6. Ausgabe der Filterkriterien und optional Tabelle.
"""
def get_rows_by_value(column, value, file=cache_file, return_table=False):
    header, lines = get_content(file)
    # Existenz der Spalte prüfen
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return
    col_index = header.index(column)
    rows = []
    # Alle Datenzeilen durchgehen
    for line in lines[1:]:
        row = line.strip().split(",")
        # Prüfen, ob der Zellenwert in der gewünschten Liste ist
        if row[col_index] in value:
            rows.append(row)
    # Gefilterte Zeilen in Cache schreiben
    generate_cache_file(header, rows)
    # Ergebnis anzeigen
    print(f"\nOnly returned rows with value in column: {column}")
    print(f"For Values: {value}")
    if return_table:
        print_formatted_table()

"""
Sortiert die Tabelle nach einer bestimmten Spalte mithilfe des Bubble-Sorts.

Parameter:
  column (str): Spaltenname zum Sortieren.
  area (list of int): Zwei Werte [Start, Ende] der Zeilenindizes.
  reverse (bool): Aufsteigend (False) oder absteigend (True).
  file (str): Eingabe-/Ausgabedatei.
  return_table (bool): Wenn True, wird die sortierte Tabelle angezeigt.

Quelle & Algorithmus:
  Bubble Sort – einfacher, aber ineffizient für große Datenmengen.
  Erklärung: https://www.geeksforgeeks.org/bubble-sort/
"""
def sort_by(column, area, reverse=False, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print(f"Spalte '{column}' nicht gefunden.")
        return
    # Bereichsanpassung: Wenn Ende -1, bis zum Schluss
    if not area or area[1] < 0:
        area = [area[0], len(lines)]
    # Teildaten einlesen
    rows = [line.strip().split(",") for line in lines[area[0]:area[1]]]
    col_index = header.index(column)
    # Bubble Sort Implementierung
    for i in range(len(rows)):
        for j in range(len(rows) - i - 1):
            a = rows[j][col_index]
            b = rows[j + 1][col_index]
            # Vergleiche Strings oder Zahlen (hier als Strings)
            if (a > b and not reverse) or (a < b and reverse):
                rows[j], rows[j + 1] = rows[j + 1], rows[j]
    # Sortierte Daten zurückschreiben und optional anzeigen
    generate_cache_file(header, rows)
    print(f"\nSorted by column: {column}")
    print(f"Range: {area[0]} to {area[1]}")
    if return_table:
        print_formatted_table()

"""
Zählt die Gesamtanzahl der 'Menschen' in der Tabelle.

Parameter:
  file (str): Pfad zur Eingabedatei.

Rückgabe:
  total_humans (int): Summe aller Werte in Spalte 'Number'.
"""
def count_humans(file=cache_file):
    header, lines = get_content(file)
    col_index = header.index("Number")
    total_humans = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        total_humans += int(row[col_index])
    return total_humans

"""
Berechnet das Durchschnittsalter der 'Menschen' in der Tabelle.

Parameter:
  file (str): Pfad zur Eingabedatei.

Rückgabe:
  avg_age (float): Durchschnittsalter basierend auf dem aktuellen Jahr (2025).
"""
def average_age(file=cache_file):
    header, lines = get_content(file)
    col_index = header.index("YearOfBirth")
    current_year = 2025
    total_age = 0
    count = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        age = current_year - int(row[col_index])
        total_age += age
        count += 1
    if count == 0:
        print("No data available to calculate average age.")
        return None
    return total_age / count

# ===== MAIN FUNCTION =====
# Steuert den Programmablauf: Einlesen, Filtern, Anzeigen, Sortieren, Statistiken

def main():
    cache_file = "cache.csv"       # Pfad zur Cache-Datei
    original_file = "names.csv"    # Ursprüngliche Namensliste
    source_file = original_file    # Aktuelle Datenquelle (wird beim Filtern angepasst)
    while True:
        # Nutzerabfrage: Filter, Statistik, Sortierung, Reset oder Beenden
        user_choice = input("\nMöchten Sie Daten filtern, Statistiken sehen, sortieren, zurücksetzen oder beenden? (filter/stats/sort/reset/quit): ").lower()
        if user_choice == 'quit':
            print("Programm wird beendet...")
            break
        elif user_choice == 'filter':
            print("Verfügbare Spalten: StateCode, YearOfBirth, Sex, Number")
            column = input("Spalte zum Filtern: ")
            values = input("Gewünschte Werte (kommagetrennt): ").split(',')
            values = [v.strip() for v in values]
            get_rows_by_value(column, values, source_file)
            source_file = cache_file
        elif user_choice == 'stats':
            print(f"Gesamtanzahl: {count_humans(source_file)}")
            avg = average_age(source_file)
            if avg is not None:
                print(f"Durchschnittsalter: {avg:.1f} Jahre")
        elif user_choice == 'sort':
            column = input("Spalte zum Sortieren: ")
            start = int(input("Startzeile (Standard 1): ") or 1)
            end = int(input("Endzeile (Standard alle): ") or -1)
            direction = input("Absteigend sortieren? (yes/no): ").lower() == 'yes'
            sort_by(column, [start, end], direction, source_file)
        elif user_choice == 'reset':
            print("Datenquelle zurückgesetzt auf originale Datei.")
            source_file = original_file
        else:
            print("Ungültige Eingabe, bitte erneut versuchen.")

if __name__ == "__main__":
    main()
